In [1]:
import pandas as pd
import yfinance as yf
import os
from pandas_datareader import data as pdr

yf.pdr_override()

In [2]:
StocksListfilePath = "StocksList1.xlsx"
stocklist = pd.read_excel(StocksListfilePath)
stocklist = stocklist[['Symbol','RS Rating']][:10] #first 10 stocks from list
stocklist

,Symbol,RS Rating
0,INMD,99
1,ENPH,99
2,PLMR,98
3,VCTR,95
4,SEDG,98
5,GSX,99
6,ARES,95
7,PFSI,92
8,ACMR,99
9,PAAS,95


In [3]:
#Output table
selectedStocks= pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day MA", "200 Day MA", "52 Week Low", "52 week High"])

smaUsed=[50,150,200]
for _,row in stocklist.iterrows():
    #print(row)
    stock = row['Symbol']
    RS_Rating = row['RS Rating']
    print("-------------Processing for stock: ",stock)
    
    try:

        df = pdr.get_data_yahoo(stock, start="2018-01-01", end="2020-06-11")

        for sma in smaUsed:
            df["SMA_"+str(sma)]=round(df['Adj Close'].rolling(window=sma).mean(),2)

        currentClose=df["Adj Close"][-1]
        moving_average_50=df["SMA_50"][-1]
        moving_average_150=df["SMA_150"][-1]
        moving_average_200=df["SMA_200"][-1]
        low_of_52week=min(df["Adj Close"][-260:])
        high_of_52week=max(df["Adj Close"][-260:])

        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20=0

        #print(currentClose,moving_average_50,moving_average_150,moving_average_200,low_of_52week,high_of_52week,moving_average_200_20)

        cond_1 = cond_2 = cond_3 = cond_4 = cond_5 = cond_6 = cond_7 = cond_8 = False

        #Condition 1: Current Price > 150 SMA and > 200 SMA
        if(currentClose>moving_average_150>moving_average_200):
            cond_1=True

        #Condition 2: 150 SMA > 200 SMA
        if(moving_average_150>moving_average_200):
            cond_2=True

        #Condition 3: 200 SMA trending up for at least 1 month (ideally 4-5 months)
        if(moving_average_200>moving_average_200_20):
            cond_3=True

        #Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        if(moving_average_50>moving_average_150>moving_average_200):
            #print("Condition 4 met")
            cond_4=True

        #Condition 5: Current Price > 50 SMA
        if(currentClose>moving_average_50):
            cond_5=True

        #Condition 6: Current Price is at least 30% above 52 week low (Many of the best are up 100-300% before coming out of consolidation)
        if(currentClose>=(1.3*low_of_52week)):
            cond_6=True

        #Condition 7: Current Price is within 25% of 52 week high
        if(currentClose>=(.75*high_of_52week)):
            cond_7=True

        #Condition 8: IBD RS rating >70 and the higher the better
        if(RS_Rating>70):
            cond_8=True

        if(cond_1 and cond_2 and cond_3 and cond_4 and cond_5 and cond_6 and cond_7 and cond_8):
            selectedStocks = selectedStocks.append({'Stock': stock, "RS_Rating": RS_Rating, "50 Day MA": moving_average_50,
                                            "150 Day MA": moving_average_150, "200 Day MA": moving_average_200, 
                                            "52 Week Low": low_of_52week, "52 week High": high_of_52week}, ignore_index=True)   

    except Exception:
        print("No data on "+stock)    

-------------Processing for stock:  INMD
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  ENPH
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  PLMR
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  VCTR
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  SEDG
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  GSX
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  ARES
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  PFSI
[*********************100%***********************]  1 of 1 completed
-------------Processing for stock:  ACMR
[*********************100%***********************]  1 of 1 completed
-----------

In [4]:
selectedStocks

,Stock,RS_Rating,50 Day MA,150 Day MA,200 Day MA,52 Week Low,52 week High
0,PLMR,98,61.75,54.84,50.75,22.910000,82.809998
1,SEDG,98,115.16,103.62,98.75,53.259998,150.889999
2,GSX,99,35.52,31.60,27.27,9.070000,45.419998
3,ARES,95,34.29,33.98,32.45,22.709999,40.606640
4,ACMR,99,45.84,32.54,27.86,12.590000,65.690002
5,PAAS,95,22.44,21.10,20.00,10.776965,30.440001
